In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

# Downloading the data

In [2]:
%pwd
## Change it to the location that you want to download your files to.

'C:\\Users\\robin\\Downloads\\MDS\\Block 6\\DSCI 525\\525-group-01\\notebooks'

In [3]:
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "rainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text) 
files = data["files"]            
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 4.11 s
Wall time: 1min 45s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 19.5 s
Wall time: 19.7 s


# Combining data CSVs

In [16]:
%%time
## here we are using a normal python way for merging the data 

%cd rainfall/
os.chmod("combined_data.csv", 0o777)
files = glob.glob('*.csv')
if "observed_daily_rainfall_SYD.csv" in files:
    files.remove("observed_daily_rainfall_SYD.csv")

if "combined_data.csv" in files:
    os.remove("combined_data.csv")
    files.remove("combined_data.csv")
    
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("([^_]*)", file)[0])
                for file in files[:3])
              )
#df.to_csv("combined_data.csv")
with open("combined_data.csv", "w") as f:
    df.to_csv(f)

[WinError 2] The system cannot find the file specified: 'rainfall/'
C:\Users\robin\Downloads\MDS\Block 6\DSCI 525\525-group-01\notebooks\rainfall
CPU times: total: 31.1 s
Wall time: 31.2 s


In [17]:
df

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.250000,-35.000000,140.6250,142.5000,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.250000,-35.000000,140.6250,142.5000,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.250000,-35.000000,140.6250,142.5000,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.250000,-35.000000,140.6250,142.5000,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.250000,-35.000000,140.6250,142.5000,1.047658e-02,ACCESS-CM2
...,...,...,...,...,...,...
2014-12-27 12:00:00,-31.709369,-29.844118,152.8125,154.6875,1.088772e-13,AWI-ESM-1-1-LR
2014-12-28 12:00:00,-31.709369,-29.844118,152.8125,154.6875,7.857531e-02,AWI-ESM-1-1-LR
2014-12-29 12:00:00,-31.709369,-29.844118,152.8125,154.6875,3.825708e+00,AWI-ESM-1-1-LR


In [4]:
df.model.unique(), df.model.nunique()

(array(['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-ESM-1-1-LR'], dtype=object), 3)

# Load the combined CSV to memory and perform a simple EDA

## Changing `dtype` of our data. Specifically, I will only test the target column`rain` and the index column `time`, while keeping all the other columns. 

In [6]:
df.index.dtype, df['rain (mm/day)'].dtype

(dtype('O'), dtype('float64'))

In [7]:
print(f"Memory usage with float64 rain and object type time: {df.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64 rain and object type time: 252.56 MB


In [8]:
df.index = pd.to_datetime(df.index)
df["rain (mm/day)"] = df['rain (mm/day)'].astype('float32')

In [9]:
df.index.dtype, df["rain (mm/day)"].dtype

(dtype('<M8[ns]'), dtype('float32'))

In [10]:
import numpy as np
np.dtype('datetime64[ns]') == np.dtype('<M8[ns]')

True

In [11]:
print(f"Memory usage with float32 rain and datetime time/index column: {df.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float32 rain and datetime time/index column: 234.52 MB


Although this might seem insignificant, when we only use the needed columns and apply this datatype, we should see better results. Below, I'll select only the required columns and then do this process again to check.

## Load just columns that we want

In [15]:
pd.read_csv("combined_data.csv")

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.250000,-35.000000,140.6250,142.5000,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.250000,-35.000000,140.6250,142.5000,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.250000,-35.000000,140.6250,142.5000,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.250000,-35.000000,140.6250,142.5000,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.250000,-35.000000,140.6250,142.5000,1.047658e-02,ACCESS-CM2
...,...,...,...,...,...,...,...
4509955,2014-12-27 12:00:00,-31.709369,-29.844118,152.8125,154.6875,1.088772e-13,AWI-ESM-1-1-LR
4509956,2014-12-28 12:00:00,-31.709369,-29.844118,152.8125,154.6875,7.857531e-02,AWI-ESM-1-1-LR
4509957,2014-12-29 12:00:00,-31.709369,-29.844118,152.8125,154.6875,3.825708e+00,AWI-ESM-1-1-LR
4509958,2014-12-30 12:00:00,-31.709369,-29.844118,152.8125,154.6875,6.477188e+00,AWI-ESM-1-1-LR


In [18]:
%%time
df2 = pd.read_csv("combined_data.csv", usecols=['time', 'rain (mm/day)', 'model'])
df2
#print(df["model"].value_counts())

CPU times: total: 3.89 s
Wall time: 3.89 s


,time,rain (mm/day),model
0,1889-01-01 12:00:00,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,1.047658e-02,ACCESS-CM2
...,...,...,...
4509955,2014-12-27 12:00:00,1.088772e-13,AWI-ESM-1-1-LR
4509956,2014-12-28 12:00:00,7.857531e-02,AWI-ESM-1-1-LR
4509957,2014-12-29 12:00:00,3.825708e+00,AWI-ESM-1-1-LR
4509958,2014-12-30 12:00:00,6.477188e+00,AWI-ESM-1-1-LR


In [20]:
print(f"Memory usage with selected columns: {df2.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with selected columns: 108.24 MB


In [24]:
df2

,time,rain (mm/day),model
0,1889-01-01 12:00:00,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,1.047658e-02,ACCESS-CM2
...,...,...,...
4509955,2014-12-27 12:00:00,1.088772e-13,AWI-ESM-1-1-LR
4509956,2014-12-28 12:00:00,7.857531e-02,AWI-ESM-1-1-LR
4509957,2014-12-29 12:00:00,3.825708e+00,AWI-ESM-1-1-LR
4509958,2014-12-30 12:00:00,6.477188e+00,AWI-ESM-1-1-LR


Now, I'll convert the datatypes again as done before and check how much it makes a difference.

In [25]:
df2.time = pd.to_datetime(df2.time)
df2["rain (mm/day)"] = df2['rain (mm/day)'].astype('float32')

In [26]:
df2.time.dtype, df2["rain (mm/day)"].dtype

(dtype('<M8[ns]'), dtype('float32'))

In [27]:
print(f"Memory usage with selected columns and datatypes: {df2.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with selected columns and datatypes: 90.20 MB


As we can see, the memory usage change is significant compared to the original. 

## Loading in chunks

In [28]:
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2        1932840
ACCESS-ESM1-5     1610700
AWI-ESM-1-1-LR     966420
dtype: int32
